# Descriptive Statistics

In [1]:
using StatsBase, DataFrames, DataFrameMacros
using RCall, CategoricalArrays, TexTables
using Distributions, Gadfly, Compose
using MLJ: schema

In [2]:
include("pubh.jl");
@rlibrary readr
@rimport pubh

In [3]:
#| output: false
R"""
library(pubh)
library(sjlabelled)
library(readr)
""";

┌ Warning: RCall.jl: Loading required package: emmeans
│ Loading required package: ggformula
│ Loading required package: ggplot2
│ Loading required package: scales
│ Loading required package: ggridges
│ 
│ New to ggformula?  Try the tutorials: 
│ 	learnr::run_tutorial("introduction", package = "ggformula")
│ 	learnr::run_tutorial("refining", package = "ggformula")
│ Loading required package: gtsummary
│ Loading required package: huxtable
│ 
│ Attaching package: ‘huxtable’
│ 
│ The following object is masked from ‘package:gtsummary’:
│ 
│     as_flextable
│ 
│ The following object is masked from ‘package:scales’:
│ 
│     number_format
│ 
│ The following object is masked from ‘package:ggplot2’:
│ 
│     theme_grey
│ 
│ Loading required package: magrittr
└ @ RCall ~/.julia/packages/RCall/dDAVd/src/io.jl:172
┌ Warning: RCall.jl: 
│ Attaching package: ‘sjlabelled’
│ 
│ The following object is masked from ‘package:huxtable’:
│ 
│     set_label
│ 
│ The following object is masked from ‘packa

## Introduction

For understanding descriptive statistics, first we should be able to distinguish between:

1.  Big and small datasets. For our tutorials, we will consider a large data set if the number of observations is greater or equal than 100. We will define a small dataset if the number of observations is less or equal than 30.
2.  Numerical and categorical variables.
3.  Continuous and discrete variables.
4.  Continuous variables with a Normal distribution (or close enough) and those not normally distributed.

::: callout-note
We will use two datasets, `wcgs` as an example of a large dataset and `kfm` as an example of a small dataset.

:::

In [4]:
wcgs = read_rds("data/wcgs.rds") |> rcopy;
wcgs |> schema

┌──────────┬───────────────────────────────┬────────────────────────────────────
│ names    │ scitypes                      │ types                             ⋯
├──────────┼───────────────────────────────┼────────────────────────────────────
│ id       │ Count                         │ Int64                             ⋯
│ age      │ Count                         │ Int64                             ⋯
│ height   │ Continuous                    │ Float64                           ⋯
│ weight   │ Continuous                    │ Float64                           ⋯
│ sbp      │ Count                         │ Int64                             ⋯
│ dbp      │ Count                         │ Int64                             ⋯
│ chol     │ Union{Missing, Count}         │ Union{Missing, Int64}             ⋯
│ beh_pat  │ Multiclass{4}                 │ CategoricalValue{String, UInt32}  ⋯
│ ncigs    │ Count                         │ Int64                             ⋯
│ dib_pat  │ Multiclass{2}  

In [5]:
kfm = read_rds("data/kfm.rds") |> rcopy;
kfm |> schema

┌────────────┬───────────────┬──────────────────────────────────┐
│ names      │ scitypes      │ types                            │
├────────────┼───────────────┼──────────────────────────────────┤
│ no         │ Count         │ Int64                            │
│ dl_milk    │ Continuous    │ Float64                          │
│ sex        │ Multiclass{2} │ CategoricalValue{String, UInt32} │
│ weight     │ Continuous    │ Float64                          │
│ ml_suppl   │ Count         │ Int64                            │
│ mat_weight │ Count         │ Int64                            │
│ mat_height │ Count         │ Int64                            │
└────────────┴───────────────┴──────────────────────────────────┘


## Categorical variables

For categorical variables (including ordinal variables), we want to know and report the number of observations on each category. We can use `tabulate`.

In [6]:
tabulate(wcgs, :arcus)

        | Freq. | Percent |  Cum.   
------------------------------------
 Absent |  2211 |  70.101 |  70.101 
Present |   941 |  29.835 |  99.937 
missing |     2 |   0.063 | 100.000 
------------------------------------
  Total |  3154 | 100.000 |         


### Contingency Tables

In epidemiology, it is frequent to have two by two (2 $\times$ 2) tables. In those cases, the outcome is binary (e.g., disease: present/absent) and the exposure is binary too. Sometimes, it is also desirable to have $n \times$ 2 tables in which we are comparing more than two groups about a particular binary outcome. For these tutorials, we will display the exposure of interest in the columns so the percentages for each categorical variable represent prevalences.

In [7]:
tabulate(wcgs, :chd, :dib_pat)

    |        |   dib_pat   |       
    |        |  A   |  B   | Total 
-----------------------------------
chd | No CHD | 1411 | 1486 |  2897 
    |    CHD |  178 |   79 |   257 
-----------------------------------
    |  Total | 1589 | 1565 |  3154 


With `cross_table` from `R`, we can show both frequency and proportion.

In [8]:
pubh.cross_tbl(
  @select(wcgs, :chd, :dib_pat),
  by = "dib_pat"
) |> print

RObject{VecSxp}
                            ─────                                  
                   A, N = 1,589   B, N = 1,565       Overall, N =  
                                                            3,154  
        ───────────────────────────────────────────────────────────
          chd                                                      
          No CHD    1,411 (89%)    1,486 (95%)        2,897 (92%)  
          CHD         178 (11%)      79 (5.0%)         257 (8.1%)  

Column names: label, stat_1, stat_2, stat_0


::: callout-note
When the code is run directly in `R` and the data frame is labelled, then `cross_tbl` takes advantage of those labels.
:::

In [9]:
#| label: tbl-tbl1
#| output: asis
#| code-fold: true
#| tbl-cap: Distribution of coronary heart disease events (CHD) by behaviour pattern.
R"""
wcgs = read_rds("data/wcgs.rds")
wcgs |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
  ) |>
  copy_labels(wcgs) |>
  select(dib_pat, chd) |> 
  cross_tbl(by = "dib_pat") |> 
  set_font_size(10) %>% theme_pubh(2) |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()
""";

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: norm

:::{.callout-note}
## Notes for R code

- First we assign labels to variables, so we can take advantage of them in tables and plots.
- We changed the reference for our outcome as it is standard to report first cases/treatment and then controls.
- With `mutate` we are generating new variables, which happen to have the same names as the old variables, thus we replace them. By doing that, as they are new, they are no longer labelled. To preserve the labels, we use `copy_labels`.
- As we did not assign the changes to a new data frame, the changes we made were not stored and their impact would be only in the code we provided. Why is this important? Because if we perform inferential statistics, we want for controls to still be the reference.
- Within the code chunk, I used the *Quarto* option: `#| output: asis`, so the html output can be interpreted.
:::

:::{.callout-tip}
## Exercise

Generate a table for the relationship between behavioural pattern (`beh_pat`) and the presence of CHD (`chd`) from the `wcgs` dataset. Remember to display the exposure of interest in columns, so percentages represent prevalence.
:::

In [10]:
#| output: asis
#| code-fold: true
#| label: tbl-tbl2
#| tbl-cap: Distribution of behavioural pattern by coronary heart disease (CHD) event.
R"""
wcgs |>
  mutate(
    chd = relevel(chd, ref = "CHD")
  ) |>
  copy_labels(wcgs) |>
  select(beh_pat, chd) |> 
  cross_tbl(by = "beh_pat") |>
  set_font_size(10) %>% theme_pubh(2) |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()
""" |> rcopy

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th><th colspan="4" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Behaviour pattern</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space

::: callout-warning
## Question

What is your main observation from the distribution of cases?
:::

::: callout-note
## Answer

The prevalence of CHD in males in the **A** groups (\~ 11%) is about the double than the prevalence in males in the **B** groups (\~ 5%). The prevalence of CHD in the population is 8.1%.
:::

### Double stratification

There are cases when we want to present distribution of categorical variables by a exposure of interest and stratified by another categorical variable (e.g. confounder) of interest.

The following code shows how to generate those kind of tables.

In [11]:
#| code-fold: true
#| output: asis
#| label: tbl-tbl3
#| tbl-cap: Distribution of coronary heart disease events (CHD) by behaviour pattern and smoking status.
R"""
wcgs |> 
  select(chd, smoker, dib_pat) |> 
  mutate(
    chd = relevel(chd, ref = "CHD"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  tbl_strata(
    strata = smoker,
    .tbl_fun = ~ .x |>
      tbl_summary(by = dib_pat, missing = "no")
  ) |> 
  cosm_sum() |> set_font_size(10) |> 
  theme_pubh(2) |> 
  set_align(1, everywhere, "center") |> 
  add_footnote("Data from the WCGS.", font_size = 9) |>
  print_html()
""" |> rcopy

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Smoker</strong></p>
</th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Non-Smoker</strong></p>
</th></tr>
<tr>
<th style="vertical-align: top

:::{.callout-note}
## Notes for R code

- We construct the table with `tbl_strata` from `gtsummary`.
- `tbl_summary` is the function behind `cross_tbl`.
- The argument `missing = "no"` is used to not shown missing values.
- With `cosm_sum` we add cosmetics to the table and converts it to a *huxtable*.
:::

## Continuous variables

`StatsBase` offers a convenient `describe` function which you can use on a data frame to get an overview of the data:

In [12]:
describe(
  @select(kfm, {Not(1, 3)}),
  :min, :max, :mean, :median, :std
)

Row,variable,min,max,mean,median,std
,Symbol,Real,Real,Float64,Float64,Float64
1,dl_milk,4.44,10.43,7.5044,7.66,1.51151
2,weight,4.12,6.578,5.31874,5.352,0.548303
3,ml_suppl,0,590,96.0,57.5,130.22
4,mat_weight,47,80,59.96,58.0,8.38113
5,mat_height,153,185,167.44,167.0,6.51892


:::{.callout-tip}
We can pass a number of symbols to the describe function to indicate which statistics to compute for each feature:

- `mean`, `std`, `min`, `max`, `median`, `first`, `last` are all fairly self explanatory
- `q25`, `q75` are respectively for the 25th and 75th percentile,
- `eltype`, `nunique`, `nmissing` can also be used
- Other functions, e.g., `rel_dis` from the `pubh.jl` script that we loaded at the start of the tutorial.
- Function `estat` from `pubh.jl` reports number of non-missing values, mean, median, standard deviation and relative dispersion.
:::

We can also use summarize.

In [13]:
summarize(@select(kfm, {Not(1, 3)})) |>
to_ascii |>
print

           | Obs |  Mean   | Std. Dev. |  Min  |  Max   
--------------------------------------------------------
   dl_milk |  50 |   7.504 |     1.512 | 4.440 | 10.430 
    weight |  50 |   5.319 |     0.548 | 4.120 |  6.578 
  ml_suppl |  50 |  96.000 |   130.220 |     0 |    590 
mat_weight |  50 |  59.960 |     8.381 |    47 |     80 
mat_height |  50 | 167.440 |     6.519 |   153 |    185 


And with more detailed summaries:

In [14]:
summarize(@select(kfm, {Not(1, 3)}), detail=true)

           | Obs |  Mean   | Std. Dev. |  Min  |   p10   |   p25   |   p50   |   p75   |   p90   |  Max   
----------------------------------------------------------------------------------------------------------
   dl_milk |  50 |   7.504 |     1.512 | 4.440 |   5.156 |   6.555 |   7.660 |   8.428 |   9.632 | 10.430 
    weight |  50 |   5.319 |     0.548 | 4.120 |   4.633 |   4.977 |   5.352 |   5.637 |   6.030 |  6.578 
  ml_suppl |  50 |  96.000 |   130.220 |     0 |   0.000 |  16.250 |  57.500 | 103.750 | 241.000 |    590 
mat_weight |  50 |  59.960 |     8.381 |    47 |  49.000 |  55.000 |  58.000 |  64.500 |  73.100 |     80 
mat_height |  50 | 167.440 |     6.519 |   153 | 159.900 | 162.000 | 167.000 | 172.000 | 175.000 |    185 


In [15]:
labs = [
  "Breast milk intake (dl/day)",
  "Weight (kg)",
  "Maternal Weight (kg)",
  "Maternal Height (m)"
];

In [16]:
estat(
  @select(kfm, {Not(1, 3, 5)}),
  labs
)

Row,Variable,n,Median,Mean,SD,CV
,String,Int64,Float64,Float64,Float64,Float64
1,Breast milk intake (dl/day),50,7.66,7.504,1.512,0.201
2,Weight (kg),50,5.352,5.319,0.548,0.103
3,Maternal Weight (kg),50,58.0,59.96,8.381,0.14
4,Maternal Height (m),50,167.0,167.44,6.519,0.039


:::{.callout-caution}
For `Not` we define columns by number. If we want to use names, the names of the columns go inside square brackets.
:::

### Stratification

Descriptive statistics of `kfm` stratified by sex:

In [17]:
summarize_by(@select(kfm, {2:6}), :sex)

     |            | Obs |  Mean   | Std. Dev. |  Min  |  Max   
---------------------------------------------------------------
 Boy |    dl_milk |  25 |   7.952 |     1.492 | 4.910 | 10.430 
     |     weight |  25 |   5.438 |     0.570 | 4.360 |  6.578 
     |   ml_suppl |  25 | 105.200 |   143.530 |     0 |    555 
     | mat_weight |  25 |  60.400 |     8.062 |    48 |     78 
---------------------------------------------------------------
Girl |    dl_milk |  25 |   7.056 |     1.422 | 4.440 | 10.030 
     |     weight |  25 |   5.199 |     0.509 | 4.120 |  6.100 
     |   ml_suppl |  25 |  86.800 |   117.658 |     0 |    590 
     | mat_weight |  25 |  59.520 |     8.832 |    47 |     80 


In [18]:
pubh.estat(@formula(weight ~ sex), data=kfm) |> 
rcopy

LoadError: LoadError: UndefVarError: `@formula` not defined
in expression starting at In[18]:1

::: callout-tip
We can call `R` to use function `cros_tbl` from package `pubh`.
:::

In [19]:
#| code-fold: true
#| output: asis
#| label: tbl-tbl4
#| tbl-cap: Descriptive statistics of the `kfm` dataset by sex.
R"""
kfm = read_rds("data/kfm.rds")
kfm |>
  select(-1) |>
  cross_tbl(by = "sex", method=1)  |>
  set_font_size(10) |> theme_pubh(2) |>
  add_footnote("Mean (sd); n (%)", font_size = 9) |>
  print_html()
""" |> rcopy

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Sex</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: normal; border-style: 

:::{.callout-note}
Package `pubh` takes advantage of labels for tables of descriptive statistics, table of regression coefficients and plots.
:::

In [20]:
#| code-fold: true
#| output: asis
#| label: tbl-tbl5
#| tbl-cap: Descriptive statistics of the WCGS dataset by behaviour pattern.
R"""
wcgs = read_rds("data/wcgs.rds")
wcgs |>
  select(-c(id, beh_pat, ncigs, time)) |>
  mutate(
    chd = relevel(chd, ref = "CHD"),
    arcus = relevel(arcus, ref = "Present"),
    smoker = relevel(smoker, ref = "Smoker")
  ) |>
  copy_labels(wcgs) |>
  cross_tbl(by = "dib_pat") |> 
  theme_pubh(2) |>
  set_font_size(10) |>
  add_footnote("n (%); Median (IQR)", font_size = 9) |>
  print_html()
""" |> rcopy

<table class="huxtable" data-quarto-disable-processing="true" style="border-collapse: collapse; border: 0px; margin-bottom: 2em; margin-top: 2em; ; margin-left: auto; margin-right: auto;  ">
<col><col><col><col><tr>
<th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th><th colspan="2" style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"><p><strong>Dichotomous behaviour</strong></p>
</th><th style="vertical-align: top; text-align: center; white-space: normal; border-style: solid solid solid solid; border-width: 0.4pt 0pt 0pt 0pt;    padding: 3pt 6pt 3pt 6pt; font-weight: bold; font-size: 10pt;"></th></tr>
<tr>
<th style="vertical-align: top; text-align: left; white-space: norm